# 施設配置問題


In [ ]:
#| default_exp gis1

In [ ]:
#| hide
from nbdev.showdoc import *

施設配置問題は，候補地の中からある目的関数を最適化するように実際に施設を配置する場所を選び出す問題である。施設配置問題を知るのに適切な文献として，[田中健一，数理最適化入門(4) : 施設配置の数理モデル(チュートリアル)，応用数理23(4),2013](https://www.jstage.jst.go.jp/article/bjsiam/23/4/23_KJ00008992858/_article/-char/ja/)が挙げられる。


施設配置問題を定めるには，需要点の集合$I$と，配置候補地$J$を定義する。そして，需要点$i \in I$から$j \in J$への移動距離$d_{ij}$が与えられるとする。さらに，需要点$i$の需要量$w_i$が与えられるとする。

また，配置候補地$j \in J$に対して0-1変数$x_j$を導入する。候補地$j \in J$に施設を配置すること1，そうでないとき0をとる変数とする。そして，$i \in I$と$j \in J$の各ペアに対して，変数$y_{ij}$を定義する。この変数は，需要$i$を施設$j$に配置するとき1，そうでないとき0となる。

施設配置問題の目的関数は，各需要点から，その需要点に割り当てられた施設への移動距離の重み付き和$\sum_{i \in I}\sum_{j \in J}w_id_{ij}y_{ij}$とする。

制約条件としては，

- 配置する施設数がちょうど$p$個となる制約:

$$
\sum_{j \in J}x_j=p
$$

- 各需要点がちょうど1つの施設に割り当てられることを課す制約

$$
\sum_{j \in J}y_{ij}=1 \ \ \forall i  \in I 
$$

- 需要点$i$が施設$j$に割り当てられるには施設$j$が配置されている必要があることを課す制約

$$
y_{ij} \leq x_j \ \ \forall i  \in I,\ \forall j  \in J 
$$

を課す。まとめると，施設配置問題は，次の整数計画問題として定式化される。

$$
\begin{array}{lll}
\min&\displaystyle \sum_{i  \in I}\sum_{j \in J}w_id_{ij}y_{ij}&\\ 
\text{s.t.}&\displaystyle\sum_{j \in J }x_j=p&\\
&\displaystyle\sum_{j\in J}y_{ij}=1&\ \forall i  \in I\\
&y_{ij}\leq\ x_j&\ \forall i  \in I\ \forall j  \in J\\
&x_j\in\{0,1\}&\ \forall j \in J\\
&y_{ij}\in\{0,1\}&\ \forall i \in I,\forall j \in J.
\end{array}
$$



この整数計画問題を解くためのプログラムを定める。まず，集合$I,J$，$w_i$，$d_{ij}$，$p$を定めるリストと辞書を定義する。

In [ ]:
#需要点の集合の定義
I=[0,1]
#配置候補地の集合
J=[0,1,2,3,4]
#需要量の定義
w={0:10,1:15,2:7,3:9,4:12}
#施設数の定義
p=2

# costsの定義
costs = {
    (0, 0): 4,
    (0, 1): 6,
    (0, 2): 9,
    (0, 3): 8,
    (0, 4): 7,
    (1, 0): 5,
    (1, 1): 8,
    (1, 2): 7,
    (1, 3): 6,
    (1, 4): 4
}

これらを用いて施設配置問題を解く関数`facilitylocation()`を定める。

In [42]:


from pulp import *

def facilitylocation(I,J,w,costs,p):
    # 空の線形計画問題を生成する
    prob = LpProblem("FacilityLocation", LpMinimize)

    # 決定変数を定める
    y=LpVariable.dicts("y",[(i,j) for i in I for j in J],cat='Binary')
    x=LpVariable.dicts("x",[j for j in J],cat='Binary')

    # 目的関数の定義
    prob += lpSum([w[i]*costs[(i, j)] * y[(i, j)] for j in J] for i in I)

    # 制約条件の定義
    prob+=lpSum([x[j] for j in J])==p

    for i in I:
        prob+=lpSum([y[(i,j)] for j in J])==1

    for i in I:
        for j in J:
            prob+=y[(i,j)]<=x[j]

    # 最適化の実行
    prob.solve();

    # Print the results
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        print(v.name, "=", v.varValue)
    print("Total Cost = ", value(prob.objective))
    


こうして定義した関数を用いて配置する施設と，各需要点の施設への割り当てを求める。

In [43]:

facilitylocation(I,J,w,costs,p)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc /var/folders/w6/pzry98n55l31dnvgymbwt9jh0000gn/T/8b46036933744d2db5bb7156cf4f5a18-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/w6/pzry98n55l31dnvgymbwt9jh0000gn/T/8b46036933744d2db5bb7156cf4f5a18-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 94 RHS
At line 108 BOUNDS
At line 124 ENDATA
Problem MODEL has 13 rows, 15 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 100 - 0.00 seconds
Cgl0004I processed model has 13 rows, 15 columns (15 integer (15 of which binary)) and 35 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 100
Cbc0038I Before mini branch and bound, 15 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and boun

この出力結果から，最適解が求まったことがわかる。配置する施設は0と4であり，需要点0は施設0に，需要点1は施設4に掘り当てることが最適であることがわかる。また，その時の目的関数値は100であることもわかる。


In [44]:
#| hide
import nbdev; nbdev.nbdev_export()